# Plotly Scroll-Zoom thourgh `ipyevents`
In following example mouse-scroll is zooming in and out the _Plotly_ chart. It is not as effective as native zoom, but it gets the job done. Code is broken down into steps for you to easy change/modify/customize it to solve your particular challenge.

## Simple Example
For the simplicity of understanding we are going to only zoom on X axis with right-most point fixed, so:
- right-most point of the cart is fixed
- left-most point is changing depending on mouse wheel movement
- we're only zooming x axis

### get some data (use yours), but...  

NOTE: In following examples X axis data is of `integer` type. If you deal with time, date, or any other - perform respective conversions.

In [60]:
import plotly.express as px
df = px.data.iris()

### plot the data

In [61]:
import plotly.graph_objects as go

In [62]:
fig = go.FigureWidget(data=go.Scatter(
    x=df.index, 
    y=df['sepal_length']))

fig

FigureWidget({
    'data': [{'type': 'scatter',
              'uid': '519cce7e-a330-473d-af7a-f7529dc18ef9',
 …

### `fig.layout.xaxis.range`

In [74]:
fig.layout.xaxis.range

(14, 149)

Tupple of indexes of left and right most points of the chart. This defines the view range. You can manualy set it by passing desired tupple. This is what we are going to change with scroll mouse event in next step.

### adding mouse scroll support by `ipyevents`
You can use any type of keyboard and mouse events on any widget.  
_**ipyevents**_: https://github.com/mwcraig/ipyevents  
Events list: https://developer.mozilla.org/en-US/docs/Web/Events

Install with conda if not installed:  
`conda install -c conda-forge ipyevents`

In [64]:
import ipyevents

Border conditions:

In [65]:
xaxis_min_range=40
# (optional) ...so you cannot zoom-in to one data point, use your taste for UX

xaxis_left_limit=0
# (optional) ...so you cannot zoom-out into negative (there is no data there)

Function to be called when mouse scroll is detected:

In [69]:
# triggered function when mouse wheel event happens
def handle_event(event):
    
    # index of left-most point of the current state of the chart
    xaxis_left=fig.layout.xaxis.range[0]
    
    # index of right-most point of the current state of the chart
    xaxis_right=fig.layout.xaxis.range[1]
    
    # setting new value by adding 'wheel movemnt`
    new_xaxis_left=xaxis_left+int(event['deltaY'])
    
    # respecting border conditions...
    if (xaxis_right-new_xaxis_left)>xaxis_min_range \
    and new_xaxis_left>xaxis_left_limit:
    
        # passing new 'zoomed' range to the chart
        fig.layout.xaxis.range=(new_xaxis_left,xaxis_right) if xaxis_left>=0 else 0

Widget Event handler:

In [76]:
# listening to mouse events while mouse on top of widget named `fig` 
Event(source=fig, watched_events=['wheel']).on_dom_event(handle_event)

(OPTIONAL) change default mouse behaviour to `pan`, instead of _"starnge"_ Plotly zoom behaviour.

In [ ]:
fig.layout.dragmode='pan'

See how it works...

In [71]:
fig

FigureWidget({
    'data': [{'type': 'scatter',
              'uid': '519cce7e-a330-473d-af7a-f7529dc18ef9',
 …

To sum-up:  

We are only wathcing for mouse events of `wheel` type in `Event` handler. In the function being called by event handler `handle_event()` we are only interested in `deltaY` key. `deltaY` is converted into integer type and simply added to left-most index `xaxis_left` of current chart. If border conditions are met - new tupple (with new left `new_xaxis_left` and old right `xaxis_right` value) is passed to the chart as `fig.layout.xaxis.range`.